In [107]:
!pip install --upgrade openai

     ---------------------------------------- 71.9/71.9 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.4
    Uninstalling openai-0.27.4:
      Successfully uninstalled openai-0.27.4


# Generate Sonnet

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import pickle
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
def diameter2(G):
    spaths = dict(nx.all_pairs_shortest_path(G))
    path = []
    diameter = 0
    for key in spaths:
        for val in spaths[key]:
            current = len(spaths[key][val])
            if current > diameter:
                diameter = current
                path = spaths[key][val]
    diameter = diameter - 1 # Because it counts the initial node, the diameter is 1 fewer
    return diameter, path

In [3]:
POSG = nx.read_gml('shakespeare_pos_3gram_fixed.gml')
sentence_start_pos_tag_freq = pickle.load(open( "sentence_start_pos_tag.p", "rb" ))
sentence_end_pos_tag_freq = pickle.load(open( "sentence_end_pos_tag.p", "rb" ))
sentence_start_to_end_pos_tag_freq = pickle.load(open( "sentence_start_to_end_pos_tag.p", "rb" ))

In [171]:
diameter2(POSG)

(12,
 ['CC RB VBP',
  'PRP RB JJ',
  'WP$ JJ NN',
  'PRP VBZ NN',
  'TO DT NN',
  'IN PRP$ JJ',
  'PRP RB VB',
  'DT NN MD',
  'VB PRP RP',
  'CC VBG VBN',
  'NNP CC NN',
  'VB VBN CC',
  'VBN VB VBN'])

In [4]:
sorted_ec = sorted(nx.degree_centrality(POSG).items(), key=lambda x: x[1], reverse=True)
for node, ec in sorted_ec[:10]:
    print(f"{node}: {ec}")

IN DT NN: 0.06757193663110249
DT JJ NN: 0.06498545101842872
IN PRP$ NN: 0.04623343032654381
NN IN NN: 0.04591011962495959
DT NN NN: 0.03718073068218558
IN JJ NN: 0.035240866472680245
JJ NN NN: 0.0345942450695118
DT NN IN: 0.03427093436792758
IN DT JJ: 0.0316844487552538
NN IN DT: 0.028451341739411575


In [5]:

start_total_freq = sum(sentence_start_pos_tag_freq.values())
sentence_start_pos_tag_prob = {k: v/start_total_freq for k, v in sentence_start_pos_tag_freq.items()}

for k, v in sentence_start_pos_tag_freq.items():
    sentence_start_pos_tag_prob[k] = v / start_total_freq
    

In [6]:

end_total_freq = sum(sentence_end_pos_tag_freq.values())
sentence_end_pos_tag_prob = {k: v/end_total_freq for k, v in sentence_end_pos_tag_freq.items()}

for k, v in sentence_end_pos_tag_freq.items():
    sentence_end_pos_tag_prob[k] = v / end_total_freq
    

In [7]:
sentence_start_to_end_pos_tag_freq.pop(('', '')) # remove this pairing from dict to prevent being picked as a source and target

194

In [8]:

start_to_end_total_freq = sum(sentence_start_to_end_pos_tag_freq.values())
sentence_start_to_end_pos_tag_prob = {k: v/start_to_end_total_freq for k, v in sentence_start_to_end_pos_tag_freq.items()}

for k, v in sentence_start_to_end_pos_tag_freq.items():
    sentence_start_to_end_pos_tag_prob[k] = v / start_to_end_total_freq
    

In [244]:
sentence_start_to_end_pos_tag_prob

{('WRB PRP VBP', 'VBZ DT NN'): 0.0003957261574990107,
 ('CC VB DT', 'IN JJ NN'): 0.0003957261574990107,
 ('WRB PRP VBD', 'NN IN JJ'): 0.0003957261574990107,
 ('CC JJ NN', 'NN IN JJ'): 0.0003957261574990107,
 ('WRB JJ NNS', 'NNS IN NNS'): 0.0003957261574990107,
 ('WDT RB IN', 'VB DT NN'): 0.0003957261574990107,
 ('CC VB JJ', 'RP IN NNS'): 0.0003957261574990107,
 ('NNP IN DT', 'CC RB VB'): 0.0003957261574990107,
 ('RB IN JJ', 'PRP NN VBP'): 0.0003957261574990107,
 ('DT NN IN', 'NN MD VB'): 0.0003957261574990107,
 ('IN NNS CC', 'VBP PRP VB'): 0.0003957261574990107,
 ('CC VB RB', 'VBP NNS VB'): 0.0003957261574990107,
 ('CC NN NN', 'MD VB NN'): 0.0003957261574990107,
 ('NNP NN TO', 'VBZ JJ NN'): 0.0003957261574990107,
 ('RB RB PRP', 'PRP VBZ JJ'): 0.0003957261574990107,
 ('CC PRP PRP$', 'TO VB MD'): 0.0003957261574990107,
 ('NNP NNP VBD', 'IN JJ NN'): 0.0003957261574990107,
 ('NNP VBD VB', 'PRP$ NN RB'): 0.0003957261574990107,
 ('CC NN NN', 'RB VB JJ'): 0.0003957261574990107,
 ('IN JJ NN', 

In [9]:
sum(sentence_start_pos_tag_prob.values())

1.000000000000017

In [10]:
sum(sentence_end_pos_tag_prob.values())

1.0000000000000153

In [11]:
sum(sentence_start_to_end_pos_tag_prob.values())

1.0000000000000409

In [10]:
print("Unique Sentence Starts: " , len(sentence_start_pos_tag_prob.keys()))
print("Unique Sentence Endings: " , len(sentence_end_pos_tag_prob.keys()))
print("Unique Sentence Start | End: " , len(sentence_start_to_end_pos_tag_prob.keys()))

Unique Sentence Starts:  1045
Unique Sentence Endings:  786
Unique Sentence Start | End:  2421


In [11]:
print("POSG Nodes =",len(POSG.nodes()),", POSG Edges = ",len(POSG.edges()))

POSG Nodes = 3094 , POSG Edges =  7235


In [52]:
'IN PRP VBP' in POSG.nodes()

True

In [53]:
'VBG NN RB' in POSG.nodes()

True

In [55]:
nx.has_path(POSG, 'IN PRP VBP', 'VBG NN RB')

True

In [68]:
POSG.get_edge_data('WRB PRP VBP', 'VB DT NN', default=None)

{'weight': 1}

In [56]:
for path in nx.all_shortest_paths(POSG, 'IN PRP VBP', 'VBG NN RB', weight='weight'):
    print(path)

['IN PRP VBP', 'RB IN DT', 'JJ NN IN', 'VBG NN RB']


In [240]:
def gen_sonnet():
    sonnet = []
    lines_in_sonnet = 14

    for l in range(lines_in_sonnet):

        source = ''
        target = ''

        break_while = False

        while not break_while:
            pair = T
            source = pair[0][0]
            target = pair[0][1]

            path_weight = []

            for path in nx.all_simple_paths(POSG, source=source, target=target, cutoff=3):
                weight = nx.path_weight(POSG, path, weight='weight')
                path_weight.append((path, weight))

            sorted_path_weight = sorted(path_weight, key=lambda x: x[1], reverse=True)

            if len(sorted_path_weight) != 0:

                # print('checkpoint')
                # print(nx.has_path(POSG, source, target))
                # print(len(sorted_path_weight[0][0]))
                # print(source != '' and target != '')
                # print(pair[0] in sentence_start_to_end_pos_tag_freq.keys())

                if nx.has_path(POSG, source, target) == True \
                    and len(sorted_path_weight[0][0]) == 3 \
                    and source != '' and target != '' \
                    and pair[0] in sentence_start_to_end_pos_tag_freq.keys():

                    # print(' '.join(sorted_path_weight[0][0])+', '+str(len(sorted_path_weight[0][0])))
                    print(' '.join(sorted_path_weight[0][0]))
                    sonnet.append(' '.join(sorted_path_weight[0][0]))

                    break_while=True
    
    return sonnet



In [235]:
sonnet_openai = [x + ' ->' for x in sonnet]

In [236]:
sonnet_openai

['PRP$ NN WRB PRP VBD NNS IN DT NN ->',
 'IN DT NN IN JJ NN NN VB VBN ->',
 'NNS CC IN JJ NN JJ NN JJ NN ->',
 'DT NNS WDT VBD DT JJ NNS TO VB ->',
 'NNP VBZ RB RBR JJ IN PRP$ NNS JJ ->',
 'PRP VBZ CC VBZ PRP$ NNS DT NN NNS ->',
 'NNS CC JJ NNS TO NNS CC TO NNS ->',
 'NNP VBZ RB RBR JJ IN PRP$ NNS JJ ->',
 'NNP VBP JJ IN PRP RB VBP IN PRP ->',
 'CC RB IN PRP$ NN PRP VBZ PRP$ NNS ->',
 'NNP NN NNS VBP DT NN WDT VBZ PRP ->',
 'WRB DT PRP VBD CC DT JJ JJ NN ->',
 'WRB IN DT NN VB PRP RB VB RB ->',
 'PRP$ NN VBZ JJ CC NNP TO PRP VB ->']

# OpenAI Eminem

In [227]:
import os
import openai
import json

In [228]:
openai.api_key = 'INSERT OPENAI API KEY HERE'

In [229]:
for i in openai.FineTune.list()['data']:
    print(i['fine_tuned_model'])

ada:ft-personal:hatespeech-counterspeech-2023-04-25-00-13-31
curie:ft-personal-2023-04-25-01-29-23
None
None
None
ada:ft-personal:eminem-v2-2023-04-25-04-53-42
None
None
davinci:ft-personal:hs-cs-v2-2023-04-25-16-06-31
curie:ft-personal:eminem-v3-2023-04-25-15-58-28
curie:ft-personal:eminem-sonnet-2023-05-02-02-34-06


In [230]:
ft_model = 'curie:ft-personal:eminem-v3-2023-04-25-15-58-28'
# ft_model = 'curie:ft-personal:eminem-sonnet-2023-05-02-02-34-06'

In [237]:
def gen_lyrics(model, prompt, max_tokens=20, stop=['->', '\n', '=>'], temp=0.3, pres_pen=0, freq_pen=0, best_of=1, n=1):
    res = openai.Completion.create(model=model, prompt=prompt, max_tokens=max_tokens, \
                                   stop=stop, temperature=temp, \
                                   presence_penalty=pres_pen, frequency_penalty=freq_pen, \
                                   best_of=best_of, n=n)
    return res

In [242]:
sonnet_openai = [x + ' ->' for x in gen_sonnet()]

CC TO VB NNS IN DT NN IN NNS
CC PRP VBZ JJ CC RB VBD PRP VB
NNP PRP JJ NN IN IN RB VBN IN
PRP VBD PRP$ NN NN CC RB TO PRP
NNS CC IN JJ NN JJ NN JJ NN
RB PRP$ NN NN WDT NNS RB DT VBP
WP NN TO VB WRB NN IN NN NN
RB JJ TO VB CC JJ CC MD VB
NNP NNP PRP$ NN CC EX DT JJ NNS
WRB PRP VBZ DT NNS CC IN WP NN
CC NNP RB JJ NN DT RBR JJ RB
WP VBZ NN NN IN VBG WRB NN MD
CC IN RB PRP VBD CC CD NN NN
IN PRP JJ IN VBZ PRP$ NN IN PRP


In [243]:
for prompt in sonnet_openai:
    synthetic_lyric = gen_lyrics(ft_model, prompt, max_tokens=20, temp=0.3, pres_pen=0.6, freq_pen=0, best_of=3, n=1)
    synthetic_lyric = synthetic_lyric['choices'][0]['text']
    
    prompt_length = len(prompt.split(' ')[:-1])
    synthetic_lyric_length = len(synthetic_lyric.split(' ')[1:])
    
    lyric_tags = nltk.pos_tag(synthetic_lyric.split(' ')[1:])
    tags = ' '.join([i[1] for i in lyric_tags]) + ' ->'
    
    # while prompt_length != synthetic_lyric_length and tags != prompt:
    while tags != prompt:
        synthetic_lyric = gen_lyrics(ft_model, prompt, max_tokens=20, temp=0.3, pres_pen=0.6, freq_pen=0, best_of=3, n=1)
        synthetic_lyric = synthetic_lyric['choices'][0]['text']
        synthetic_lyric_length = len(synthetic_lyric.split(' ')[1:])
        
        lyric_tags = nltk.pos_tag(synthetic_lyric.split(' ')[1:])
        tags = ' '.join([i[1] for i in lyric_tags]) + ' ->'
        
    
    
    # print(prompt, synthetic_lyric, prompt_length, synthetic_lyric_length)
    print(tags, synthetic_lyric)
    print()

CC TO VB NNS IN DT NN IN NNS ->  And to do things with a bunch of women

CC PRP VBZ JJ CC RB VBD PRP VB ->  And it ain't fair but still did it happen



KeyboardInterrupt: 

In [ ]:
"Took her in my car just for her own
 For saying it I can be Shady you too
 When you're livin life's a joke too to be
 Back in my day man your parents are peasants
 In the back of that fuckin van gettin raped
 What kind of shit you not to believe in me
 Who knows maybe we get together to reminiscing
 You can just sit there when you could be
 Who tryin to act how they'd like if they're
 For saying it I can make Hailie you too
 It makes my skin and hair now it grows
 I'm gonna do things that are gonna shock ya
 With a whole lot of this shit in store
 If that means I can get Hailie back then"

In [ ]:
PRP IN PRP$ NN NN PRP VBZ IN NN ->  It in my memory bank it stays on replay

VB DT TO VB PRP VBP PRP$ NN RB ->  Get this to go I want your number privately

DT NN IN NN CC VBD DT RB NN ->  The heart of gold but hooked a maybe floozy

NNS IN DT JJ NN IN NN TO NN ->  People with no prior record of violence to property

WP VBD JJ RB RBR IN JJ NN IN ->  What was last now’s more like adder’s root in

IN JJ PRP$ NN VBZ VBN IN JJ NN ->  In brief my penitence is paid in full bye-bye

CC RB VBD DT NN IN JJ JJ NNS ->  And then popped a couple of clean healthy ones

CC PRP TO WP NN NN PRP RB VBG ->  And me to who man it's you always worrying

CC IN DT NN WP$ NN VBZ VBG VB ->  But by the time whose mind is slipping slow

NNP RP IN JJ NNS RB IN PRP$ NN ->  Jesus down with little brothers back at his house

PRP VBP PRP$ NN IN VBG NNP PRP NN ->  I slits her throat while choking Hailie me name

TO PRP$ JJ NN IN IN PRP JJS JJ ->  To my left side of at it most extreme

IN WRB PRP VBD DT NN VBD VBN PRP ->  Like how they put a rat got killed it

CC RB PRP VBP CC NN VBD JJ NN ->  And sometimes I wonder and everything seemed fine yeah

In [ ]:
CC NNS WDT IN DT JJ NNS VBD NN ->  And fucks that at the first signs turned slut

DT NN MD VB PRP RP CC VBG VBN ->  No one could get me off and being raised

RB VBP DT NNS CC VB DT NNS JJ ->  Now turn the lights and extinguish the flames low

IN WRB PRP VBD DT NN VBD VBN PRP ->  While how I got this beat was stolen it

NNP NN TO VB DT NN VBZ CC NN ->  Time's comin to send this world turns and time

CC VBZ PRP RB VBP DT NNS PRP VBP ->  But does he even know the words I say

IN NN IN PRP NN IN WP PRP VBZ ->  After school after it time for what it is

WRB NNS VBP VBG NN VBG DT NN NN ->  When hoes are jacking dude getting a hand job

DT NN MD VB PRP RP CC VBG VBN ->  No one can bring us down but being hated

VBN IN CC IN JJ NN NNS VBP RP ->  Stuck in and like real music fans it's up

RB JJ IN NNS IN JJ NN VBD VB ->  Too many of em like last year did do

IN PRP VBZ PRP$ NNS VBP IN DT JJS ->  If he gets his hands dirty with the best

DT NNS WDT VBD DT JJ NNS TO VB ->  These beats that were the only things to rap

RB TO VB NNS RB VBP DT CD NN ->  Just to hear voices again hear a 5 year-old

In [ ]:
CC RB NNP VB NN TO VB PRP JJ ->  But so I'ma prove em to be they wrong

NNP PRP IN PRP$ NN RB IN PRP$ NN ->  I'll you out your misery always at your beck-and-call

CC PRP TO WP NN NN PRP RB VBG ->  But you're to what extent it's you not knowing

RB TO VB NNS RB VBP DT CD NN ->  Just to hear voices again it's the one thing

DT PRP MD VB PRP DT JJ NN NN ->  That I can give you the same amount uh

WRB CD CC VBZ RB JJ RB DT NN ->  Why one but it's so easy just a trick

PRP$ NN VBZ PRP IN PRP JJR CC JJR ->  My name is it before me more and more

PRP$ NNS VBG CD NNS IN JJ DT NN ->  My feet burning 10 miles in barefoot each day

NNP NNS VBP VB DT VBG NNS IN NNP ->  Motherfuckin rappers gotta stop these motherfucking threats against Bush

IN VBG PRP PRP MD VB NNP PRP RB ->  For telling him I would be Hailie you still

NNS IN IN NN VBP JJ IN IN NN ->  Things about about rap are true about about rap

CC VBZ PRP RB VBP DT NNS PRP VBP ->  And says I never write no rhymes I guess

RB PRP$ NN NN WDT NNS RB DT VBP ->  So my reputation's murder that people just all know

CC RB PRP VBP CC NN VBD JJ NN ->  But now I know and it's was wrong yeah

In [ ]:
IN PRP VBZ IN PRP PRP VBP PRP JJ ->  If it wasn't for me you see me different

NNP VBP RB VB IN NN NN RB VB ->  Don't do nothin that's beyond reason cause then there's

NNP NNP PRP$ NN CC EX DT JJ NNS ->  That's Bizzy his name but there some other people

CC MD VB IN PRP$ IN PRP VBD UH ->  And would be on my like I said Oh

VBG NN PRP VBZ NN VBZ IN PRP$ NN ->  Pissing blood he doesn't care is that his heart

PRP VBD PRP$ NN IN RB VBZ IN NN ->  I cut my hair because now is on fire

NNP VBP TO NNS IN PRP VBP RB IN ->  I'll say to women that I love em though

WDT VBZ RB JJ IN NNS VBZ DT NN ->  Which is not fuckin with rappers is a fact

VBG NN PRP VBZ NN VBZ IN PRP$ NN ->  Getting sicker he says life is like my dick

DT PRP$ NN NN VBZ DT NN VBP CD ->  All my life everyone tells a story have one

CC VBZ PRP RB VBP DT NNS PRP VBP ->  But does she even know the words I say

WP VBZ IN DT NNS VBP IN PRP$ NNS ->  Who knows if these people are in our heads

PRP VBD RB VBN NN NN CC VBD IN ->  I was always told you're brainy but was that

PRP VBP VB PRP$ IN PRP VBP PRP VBZ ->  You gotta believe her if you think she lies

In [ ]:
PRP$ NN WRB PRP VBD NNS IN DT NN ->  My mom when she heard footsteps in the hallway

IN DT NN IN JJ NN NN VB VBN ->  With the weight of public opinion shouldn't be allowed

NNS CC IN JJ NN JJ NN JJ NN ->  Stacks and in full effect full effect full effect

DT NNS WDT VBD DT JJ NNS TO VB ->  The ones that had no other choices to make

NNP VBZ RB RBR JJ IN PRP$ NNS JJ ->  Cause you're barely more toxic than your words uh-huh

PRP VBZ CC VBZ PRP$ NNS DT NN NNS ->  He sneaks and takes his parents each other's spouses

NNS CC JJ NNS TO NNS CC TO NNS ->  Drugs and white rags to riches and to piles

NNP VBZ RB RBR JJ IN PRP$ NNS JJ ->  Shit is even more empty than my pockets full

NNP VBP JJ IN PRP RB VBP IN PRP ->  I'ma make sure that I never forget about you

CC RB IN PRP$ NN PRP VBZ PRP$ NNS ->  And just in your face it takes your girls

NNP NN NNS VBP DT NN WDT VBZ PRP ->  Murderer murderer murderers murder the beat that kills me

WRB DT PRP VBD CC DT JJ JJ NN ->  When all I was and a little white girl

WRB IN DT NN VB PRP RB VB RB ->  Where at the party be It just ain't there

PRP$ NN VBZ JJ CC NNP TO PRP VB ->  My name is ice and I'm to you extreme